# 이웃을 고려한 CF

- 이웃(neighbor)을 전체 사용자로 하는 대신에 유사도가 높은 사람만을 이웃으로 선정해서 이웃의 크기를 줄이는 방법이 있다. 이렇게 대상 사용자와 유사도가 높은 사람의 평가만 사용하면 당연히 예측의 정확도가 올라갈 것으로 예상핼 볼수 있다 
- 또 하나 고려할 사항을 이웃을 정하는 기준이다. 크게 두가지 방법이 존재하는데, 하나는 이웃의 크기를 미리 정해놓고 추천 대상 사용자와 유사한 K명을 선택하는 "K Nearest Neighbors(KNN)" 방법이고, 또 다른 하나는 크기 대신 유사도의 기준 (예를 들면 상관계수 0.8 이상)을 정해 놓고 이 기준을 충족시키는 사용자를 이웃으로 정하는 Thresholding 이다 
- KNN 방법은 K라는 이웃의 크기를 미리 정하고 상위 K명을 이웃으로 정한다 
- Thresholding 에서는 유사도가 기준(S)을 넘는 사용자를 숫자에 관계없이 선택한다.
- Thresholding 방법이 KNN보다 정확하기는 하지만 정해진 기준을 넘는 사용자가 없어서 추천하지 못하는 경우가 있기 때문에 KNN이 무난하게 많이 쓰인다 

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('./data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# train, test 데이터 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs]) # change
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [3]:
# train set 사용자들의 Cosine similarities 계산
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

### 이웃을 고려한 CF 알고리즘

In [4]:
# Neighbor size를 정해서 예측치를 계산하는 함수 
def cf_knn(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_matrix:
        sim_scores = user_similarity[user_id].copy() # 현재 사용자와 다른 사용자 간의 similarity 가져오기
        movie_ratings = rating_matrix[movie_id].copy() # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        movie_ratings = movie_ratings.drop(none_rating_idx) # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        sim_scores = sim_scores.drop(none_rating_idx) # 현재 영화를 평가하지 않은 사용자의 similarity값 제거
##### (2) Neighbor size가 지정되지 않은 경우        
        if neighbor_size == 0:          
            mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum() # 현재 영화를 평가한 모든 사용자의 가중평균값 구하기
##### (3) Neighbor size가 지정된 경우
        else:                       
            # 해당 영화를 평가한 다른 사용자 수가 매우 작으면 계산에 문제가 생기기 때문에 
            # 해당 영화를 평가한 사용자가 최소 2명이 되는 경우에만 계산 
            if len(sim_scores) > 1: 
                neighbor_size = min(neighbor_size, len(sim_scores)) # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                sim_scores = np.array(sim_scores) # array로 바꾸기 (argsort를 사용하기 위함)
                movie_ratings = np.array(movie_ratings)
                user_idx = np.argsort(sim_scores) # 유사도를 순서대로 정렬
                sim_scores = sim_scores[user_idx][-neighbor_size:] # 유사도를 neighbor size만큼 받기
                movie_ratings = movie_ratings[user_idx][-neighbor_size:] # 영화 rating을 neighbor size만큼 받기
                mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum() # 최종 예측값 계산 
            else:
                mean_rating = 3.0
    else:
        mean_rating = 3.0
    return mean_rating

# 정확도 계산
score(cf_knn, neighbor_size=30)

1.0183339813575383

### 주어진 사용자에 대해 추천을 받기

In [5]:
# 전체 데이터로 full matrix와 cosine similarity 구하기 
# 실제 추천을 할 때는 train, test set을 나눌 필요가 없기 때문에 전체 데이터를 사용해서 full matrix와 사용자 간 유사도 (user_similarity)를 구한다
rating_matrix = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')

matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

# 사용자 ID, 추천 아이템 수, 이웃 크기를 넘겨 받아서 주어진 사용자에 대한 아이템을 돌려주는 함수 
def recom_movie(user_id, n_items, neighbor_size=30):
    user_movie = rating_matrix.loc[user_id].copy() # 현 사용자의 평점 데이터만 골라서 User_movie에 복사 
    for movie in rating_matrix:
        # 현 사용자가 이미 평가한 영화는 제외 (평점을 0으로)        
        if pd.notnull(user_movie.loc[movie]):
            user_movie.loc[movie] = 0
        # 현 사용자가 평가하지 않은 영화의 예상 평점 계산
        else:
            user_movie.loc[movie] = cf_knn(user_id, movie, neighbor_size)
    # 영화를 예상 평점에 따라 정렬해서 제목을 뽑아서 돌려 줌
    movie_sort = user_movie.sort_values(ascending=False)[:n_items] # 내림차순으로 상위의 영화를 추천 
    recom_movies = movies.loc[movie_sort.index]
    recommendations = recom_movies['title']
    return recommendations

# 2번 사용자에 대해서 5개의 추천 영화를 CF 알고리즘 (이웃 크기 30)으로 설정 
recom_movie(user_id=2, n_items=5, neighbor_size=30)

movie_id
1293                     Ayn Rand: A Sense of Life (1997)
1500    Prisoner of the Mountains (Kavkazsky Plennik) ...
1189                              That Old Feeling (1997)
1467                                     Cure, The (1995)
318                       Everyone Says I Love You (1996)
Name: title, dtype: object

### 최적의 neighbor size 구하기
- 다음으로 결정할 것은 이웃 크기이다. 일반적으로 이웃 크기가 증가하면서 추천 정확도는 증가하다가 어느 시점 이후에는 감소하는 형태를 보인다. 즉 추천의 정확도를 최대로 하는 최적의 이웃 크기가 존재하는 것이 보통이다.
- 만일 이웃 크키가 지나치게 커지만 개인의 취향이 반영되지 않는다. CF의 기본 가정이 취향이 비슷한 사람의 집단이 존재하는 것이데, 이웃 크기를 너무 크게 하면(극단적인 경우가 전체 사용자를 이웃으로 사용하는 경우이다) 집단별 취향의 차이가 없어지게 된다.
- 따라서 이웃 크기가 어느 기준 이상 커지면 정확도도 떨어지게 된다. 반대로 이웃 크기가 지나치게 작으면 현재 사용자와 취향(평가 패턴)의 유사도가 매우 높은 소수의 이웃의 평가만을 사용하게 된다. 이렇게 되면 소수의 평가에 지나치게 의존하게 되어 예측치의 신뢰성이 낮아지게 된다. 이러한 문제를 과접합(over-fitting)이라 한다. 

In [6]:
# train set으로 full matrix와 cosine similarity 구하기 
rating_matrix = x_train.pivot_table(values='rating', index='user_id', columns='movie_id')

matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)
for neighbor_size in [10, 20, 30, 40, 50, 60]:
    print("Neighbor size = %d : RMSE = %.4f" % (neighbor_size, score(cf_knn, neighbor_size)))

Neighbor size = 10 : RMSE = 1.0351
Neighbor size = 20 : RMSE = 1.0199
Neighbor size = 30 : RMSE = 1.0183
Neighbor size = 40 : RMSE = 1.0188
Neighbor size = 50 : RMSE = 1.0194
Neighbor size = 60 : RMSE = 1.0200


- 위 결과에 따라 RMSE를 최소화하는 최적의 이웃 크기는 대략 30 부근임을 알 수 있다. 